In [16]:
import sys
sys.path.append('../')

In [17]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, Table, text

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

In [18]:
USER_DB = POSTGRES_UTEA['USER']
PASS_DB = POSTGRES_UTEA['PASSWORD']
HOST_DB = POSTGRES_UTEA['HOST']
PORT_DB = POSTGRES_UTEA['PORT']
NAME_DB = POSTGRES_UTEA['DATABASE']

ENGINE = create_engine(f'postgresql+psycopg://{USER_DB}:{PASS_DB}@{HOST_DB}:{PORT_DB}/{NAME_DB}')

PATH_GRUPOS_COSECHA = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPO_COSECHA.xlsx'
PATH_INSTITUCIONES = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\INSTITUCIONES.xlsx'

PATH_PLAN_ZAFRA = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\COMPROMISOS ZAFRA 2025 - Documents\PLAN ZAFRA 2025\PLAN DE ZAFRA OFICIAL.xlsx'
PATH_LAICA = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\COOR_GERENCIA_CANA - PARTES_LAICA\LAICA 6_2025.xlsx'
PATH_GRUPOS_ECO = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPOS ECONOMICOS.xlsx'
PATH_COMPROMISO_FIRMADO = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\COMPROMISOS FIRMADOS ZAFRA.xlsx'
PATH_TCH = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\TCHS.xlsx'

PATH_XLSX_ACCESOS_AGROCITTCA = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AGRO-CITTCA\LISTA.xlsx'


In [19]:
def cargar_codigos_cosecha():
    try:
        df = pd.read_excel(PATH_GRUPOS_COSECHA, sheet_name='2025')
        df = df[['INS', 'CODIGO CAÑERO', 'NOMBRE CAÑERO', 'GRUPO DE COSECHA', 'DELEGADO']]
        df = df.rename(columns={
            'INS': 'inst',
            'CODIGO CAÑERO': 'cod_ca',
            'NOMBRE CAÑERO': 'nom_ca',
            'GRUPO DE COSECHA': 'cod_cos',
            'DELEGADO': 'delegado'
        })
        df = df.sort_values(by=['cod_cos', 'inst', 'cod_ca'])
        return df
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {e}")
        return pd.DataFrame()

def cargar_cod_cos_a_db(df):
    # Convertir tipos para Postgres
    df['inst']     = df['inst'].astype('Int64')
    df['cod_ca']   = df['cod_ca'].astype('Int64')
    df['cod_cos']  = df['cod_cos'].astype('Int64')
    df['nom_ca']   = df['nom_ca'].astype(str)

    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_codigos_cosecha RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_codigos_cosecha',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de codigos cosecha")

def get_accesos_agrocittca():
    try:
        df = pd.read_excel(PATH_XLSX_ACCESOS_AGROCITTCA)
        df = df.sort_values(by=['cod_cos'])
        return df
    except Exception as e:
        print(f"❌ Error accesos agrocittca: {e}")
        return pd.DataFrame()

def cargar_accesos_agrocittca_a_db(df):
    # Convertir tipos para Postgres
    df['id']     = df['id'].astype('Int64')
    df['cod_cos']  = df['cod_cos'].astype('Int64')
    df['cod_ca']   = df['cod_ca'].astype('Int64')
    df['nom_ca']   = df['nom_ca'].astype(str)

    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        #conn.execute(text(f'TRUNCATE TABLE data.data_accesos_agrocittca RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_accesos_agrocittca',
            con=conn,  # conexión cruda dentro de la transacción
            schema='data',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de acceos de agrocittca")


def cargar_instituciones():
    try:
        df = pd.read_excel(PATH_INSTITUCIONES)
        return df
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {e}")
        return pd.DataFrame()

def cargar_instituciones_a_db(df):
    # Convertir tipos para Postgres
    df['cod']     = df['cod'].astype('Int64')
    df['nom_inst']   = df['nom_inst'].astype(str)
    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_instituciones RESTART IDENTITY'))
        # Insertar nuevos datos
        df.to_sql(
            name='data_instituciones',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de instituciones")

def cargar_plan_zafra():
    try:
        df = pd.read_excel(PATH_PLAN_ZAFRA, sheet_name='BASE CAÑEROS')
        df = df[['INST.', 'COD. CAÑERO.1', 'NOMBRE DEL CAÑERO', 'TONELADAS COMPROMISO']]        
        df = df.rename(columns={
            'INST.': 'inst',
            'COD. CAÑERO.1': 'cod_ca',
            'NOMBRE DEL CAÑERO': 'nom_ca',
            'TONELADAS COMPROMISO': 'compromiso'
        })
        df = df.sort_values(by=['inst', 'cod_ca'])
        return df
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {e}")
        return pd.DataFrame()

def cargar_plan_zafra_a_db(df):
    # Convertir tipos para Postgres
    df['inst']     = df['inst'].astype('Int64')
    df['cod_ca']   = df['cod_ca'].astype('Int64')
    df['nom_ca']   = df['nom_ca'].astype(str)
    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_plan_zafra RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_plan_zafra',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de plan de zafra")
    
def get_resumen_estimativa():
    try:
        query = "select unidad_01, unidad_03, categoria, sum(area) as area from catastro_iag.estimativa group by unidad_01, unidad_03, categoria"
        df = pd.read_sql(query, ENGINE)
        return df
    except Exception as e:
        print("❌ Error al leer datos de ESTIMATIVA:", e)
        return pd.DataFrame()
    return None

def cargar_resumen_estimativa_a_db(df):
    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_resumen_estimativa RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_resumen_estimativa',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de resumen estimativa")
    
def cargar_laica():
    try:
        df = pd.read_excel(PATH_LAICA, sheet_name='Datos')
        df = df[['Fecha', 'Cod. Cañero', 'Nombre Cañero', 'kg Caña Neta', '% Material Extraño (Inspección)', '% Pureza', 'PCF', 'Tipo Cosecha', 'Tipo Caña']]        
        df = df.rename(columns={
            'Fecha': 'fecha',
            'Cod. Cañero': 'cod_ca',
            'Nombre Cañero': 'nom_ca',
            'kg Caña Neta': 'kg_entregado',
            '% Material Extraño (Inspección)': 'me', 
            '% Pureza': 'pureza', 
            'PCF': 'pcf', 
            'Tipo Cosecha': 'tipo_cosecha', 
            'Tipo Caña': 'tipo_cana'
        })
        return df
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {e}")
        return pd.DataFrame()

def cargar_datos_laica_a_db(df):
    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_laica RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_laica',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de datos laica")

def cargar_grupos_eco():
    try:
        df = pd.read_excel(PATH_GRUPOS_ECO, sheet_name='RESUMEN_GRUPO_ECO')
        df = df.rename(columns={
            'DEUDOR': 'deudor',
            'CODIGO': 'codigo',
            'NOMBRE': 'nombre',
            'GRUPO ECONOMICO': 'grupo_economico',
            'COD. INSTITUCION': 'cod_institucion',
            'ORDEN': 'orden',
            'Cabeza del Grupo': 'cabeza_del_grupo'
        })
        return df
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {e}")
        return pd.DataFrame()

def cargar_datos_grupo_eco_a_db(df):
    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_grupo_eco RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_grupo_eco',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de datos grupos economicos")

def cargar_compromiso_firmado():
    try:
        df = pd.read_excel(PATH_COMPROMISO_FIRMADO, sheet_name='RESUMEN_COMPROMISO')
        df = df.rename(columns={
            'COD.': 'cod',
            'TONELADAS': 'toneladas',
            'GRUPO_ECO': 'grupo_eco'
        })
        return df
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {e}")
        return pd.DataFrame()

def cargar_compromiso_firmado_a_db(df):
    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_compromiso_firmado RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_compromiso_firmado',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de datos grupos economicos")

def cargar_estimativa_lote_a_db(df):
    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_estimativa_lote RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_estimativa_lote',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de datos estimativa lote")

def cargar_xlsx_tchs():
    try:
        df = pd.read_excel(PATH_TCH)
        return df
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {e}")
        return pd.DataFrame()

def cargar_estimativa_lote_a_db(df):
    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_estimativa_lote RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_estimativa_lote',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de datos estimativa lote")

def get_propiedades_estimativa():
    try:
        query = "select distinct(unidad_01), unidad_02 from catastro_iag.estimativa order by unidad_01"
        df = pd.read_sql(query, ENGINE)
        return df
    except Exception as e:
        print("❌ Error al leer datos de ESTIMATIVA:", e)
        return pd.DataFrame()
    return None

def get_estimativa_idd():
    try:
        query = "SELECT unidad_01, unidad_03, unidad_05, categoria, area, unidad_01 || '|' || categoria idd FROM catastro_iag.estimativa"
        df = pd.read_sql(query, ENGINE)
        return df
    except Exception as e:
        print("❌ Error al leer datos de ESTIMATIVA:", e)
        return pd.DataFrame()
    return None


In [5]:
accesos = get_accesos_agrocittca()
cargar_accesos_agrocittca_a_db(accesos)

ProgrammingError: (psycopg.errors.InvalidSchemaName) no existe el esquema «data»
LINE 2: CREATE TABLE data.data_accesos_agrocittca (
                     ^
[SQL: 
CREATE TABLE data.data_accesos_agrocittca (
	id BIGINT, 
	cod_cos BIGINT, 
	cod_ca BIGINT, 
	nom_ca TEXT, 
	usuario TEXT, 
	password TEXT, 
	propiedad TEXT, 
	correo_nax TEXT, 
	password_nax TEXT
)

]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [20]:
codigos_cosecha = cargar_codigos_cosecha()
cargar_cod_cos_a_db(codigos_cosecha)

✅ Se ha actualziado la tabla de codigos cosecha


In [7]:
instituciones = cargar_instituciones()
cargar_instituciones_a_db(instituciones)

✅ Se ha actualziado la tabla de instituciones


In [8]:
plan_zafra = cargar_plan_zafra()
cargar_plan_zafra_a_db(plan_zafra)

✅ Se ha actualziado la tabla de plan de zafra


In [9]:
resumen_estimativa = get_resumen_estimativa()
cargar_resumen_estimativa_a_db(resumen_estimativa)

✅ Se ha actualziado la tabla de resumen estimativa


In [13]:
datos_laica = cargar_laica()
datos_laica['tn_entregado'] = datos_laica['kg_entregado'] / 1000
cargar_datos_laica_a_db(datos_laica)

✅ Se ha actualziado la tabla de datos laica


In [10]:
datos_grupo_eco = cargar_grupos_eco()
cargar_datos_grupo_eco_a_db(datos_grupo_eco)

✅ Se ha actualziado la tabla de datos grupos economicos


In [11]:
datos_compromiso_firmado = cargar_compromiso_firmado()
cargar_compromiso_firmado_a_db(datos_compromiso_firmado)

✅ Se ha actualziado la tabla de datos grupos economicos


In [12]:
tch_xlsx = cargar_xlsx_tchs()
props_estimativa = get_propiedades_estimativa()
lista_cods_props_tch_xlsx = list(set(tch_xlsx['cod_prop']))
lista_cods_props_estiamtiva = list(set(props_estimativa['unidad_01']))

In [13]:
cods_eliminar_de_xlsx = [x for x in lista_cods_props_tch_xlsx if x not in lista_cods_props_estiamtiva]
print('CODIGOS A ELIMINAR DE XLSX:')
cods_eliminar_de_xlsx

CODIGOS A ELIMINAR DE XLSX:


[]

In [14]:
cods_agregar_a_xlsx = [x for x in lista_cods_props_estiamtiva if x not in lista_cods_props_tch_xlsx]
print('CODIGOS A AGREGAR A XLSX:')
cods_agregar_a_xlsx

CODIGOS A AGREGAR A XLSX:


[]

In [15]:
df_filtrado = props_estimativa[props_estimativa['unidad_01'].isin(cods_agregar_a_xlsx)]
df_filtrado

,unidad_01,unidad_02
